# <center> HW for ML-7

>Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).


Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
- Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать **F1-score**.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций **не должно превышать 50**.

In [4]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV # метод для подбора гиперпараметров
from sklearn.model_selection import RandomizedSearchCV
#делаем импорт hyperplot
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов
# импортируем optuna
import optuna

%matplotlib inline
plt.style.use('seaborn')

C:\Users\Павел\AppData\Local\Temp\ipykernel_9352\2413690768.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [5]:
# Загрузим данные и проверим их на читаемость
data = pd.read_csv('../data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [6]:

display(data.describe())

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.542255,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.498278,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Зададим вектор у и матрицу X
X = data.drop(['Activity'], axis = 1)
y = data['Activity']

# Разобъем данные на тренировочную, тестовую и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size =0.2, stratify=y, random_state =42)
#разбиваем валидационную выборку на валидационную и тестовую в соотношении 50/50
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)
print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))
print('Test shape: {}'.format(X_test.shape))


Train shape: (3000, 1776)
Valid shape: (375, 1776)
Test shape: (376, 1776)


In [8]:
# проверим распределение целевого признакак в выборках
print('Train:\n', y_train.value_counts(normalize=True), sep='')
print('Test:\n', y_test.value_counts(normalize=True), sep='')
print('Valid:\n' , y_valid.value_counts(normalize=True), sep='')

Train:
1    0.542333
0    0.457667
Name: Activity, dtype: float64
Test:
1    0.545213
0    0.454787
Name: Activity, dtype: float64
Valid:
1    0.538667
0    0.461333
Name: Activity, dtype: float64


Изначально обучим модель логистической регрессии и деревьев решения без подбора гиперпараметров, чтобы посмотреть на результат.

In [9]:
# Создадим объект класса логистической регрессии регрессии с параметрами по умолчанию
log_reg = linear_model.LogisticRegression(max_iter=50, random_state=42)
# train model
log_reg.fit(X_train, y_train)
print("accuracy на валидационном наборе: {:.2f}".format(log_reg.score(X_valid, y_valid)))
y_valid_pred = log_reg.predict(X_valid)
print('f1_score на валидационном наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

accuracy на валидационном наборе: 0.76
f1_score на валидационном наборе: 0.78


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
# создадим объект класса случайного леса
rf_ml = ensemble.RandomForestClassifier(random_state=42)
rf_ml.fit(X_train, y_train)
print("accuracy на валидационном наборе: {:.2f}".format(rf_ml.score(X_valid, y_valid)))
y_valid_pred = log_reg.predict(X_valid)
print('f1_score на валидационном наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

accuracy на валидационном наборе: 0.79
f1_score на валидационном наборе: 0.78


В целом обе модели справляются с задачей неплохо, на валидационной выборке показывают одинаковые результаты на метрики *F_1* = 0.78

In [11]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.6, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.6, 0.7, 0.9, 1]}
]
lr_model = linear_model.LogisticRegression(random_state=1, max_iter=50)
rf_model = ensemble.RandomForestClassifier(random_state=42)


In [12]:
# зададим сетку гиперпараметров
grid_search_1 = GridSearchCV(
    estimator= lr_model, 
    param_grid=param_grid, 
    cv=10, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.score(X_valid, y_valid)))
y_test_pred = grid_search_1.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))


CPU times: total: 5.86 s
Wall time: 2min 54s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.2, 'penalty': 'l1', 'solver': 'saga'}


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Проверим гиперпараметры через RandomizeSearch

In [13]:
grid_search_2 = RandomizedSearchCV(
    estimator= lr_model, 
    param_distributions=param_grid, 
    cv=10, 
    n_iter=50,
    n_jobs = -1
)  
%time grid_search_2.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_2.score(X_valid, y_valid)))
y_test_pred_2 = grid_search_2.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred_2)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: total: 5.89 s
Wall time: 1min 52s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.2}


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Проверим методы сетки и рандомного поиска на RandomForestClassifire

In [14]:
param_distributions_rf = {'n_estimators': list(range(40, 200, 40)),
              'min_samples_leaf': list(range(1, 5, 1)),
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }
grid_search_3 = GridSearchCV(
    estimator= rf_model, 
    param_grid=param_distributions_rf, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_3.fit(X_train, y_train) 
print("accuracy на валидационном наборе: {:.2f}".format(grid_search_3.score(X_valid, y_valid)))
y_valid_pred = grid_search_3.predict(X_valid)
print('f1_score на валидационном наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_3.best_params_))

CPU times: total: 14 s
Wall time: 14min 46s
accuracy на валидационном наборе: 0.79
f1_score на валидационном наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 31, 'min_samples_leaf': 2, 'n_estimators': 120}


In [15]:
grid_search_4 = RandomizedSearchCV(
    estimator= rf_model, 
    param_distributions=param_distributions_rf, 
    cv=10, 
    n_iter=50,
    n_jobs = -1
)  
%time grid_search_4.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_4.score(X_valid, y_valid)))
y_test_pred_2 = grid_search_4.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred_2)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_4.best_params_))

CPU times: total: 13.3 s
Wall time: 11min 29s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'n_estimators': 120, 'min_samples_leaf': 3, 'max_depth': 33}


Сравнив два обычных метода, можем сделать вывод о том, что метрики удалось увелисить, чуть лучше в нашем случае получилось на случайном лесе, но не значительно, если поиграться с параметрами и кросс валидацией их сожно сравнять

Так же найдем параметры при помоши методов optuna/hyperopt

In [16]:
# Зададим списки параметров для моделей
space_lr = {'C': hp.lognormal('LR_C', 0, 1.0),
       'solver': hp.choice('solver', ['liblinear', 'saga'])
       }
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

### <center> Hyperopt

In [20]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # кросс-валидация
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров

%time
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model_rf.fit(X_train, y_train)
y_train_pred = model_rf.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на валидационном наборе: {:.2f}".format(model_rf.score(X_valid, y_valid)))
y_test_pred = model_rf.predict(X_valid)
print('f1_score на валидационном наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))


CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 20/20 [08:13<00:00, 24.67s/trial, best loss: -0.8160803811393121]
Наилучшие значения гиперпараметров {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
f1_score на обучающем наборе: 0.99
accuracy на валидационном наборе: 0.79
f1_score на валидационном наборе: 0.81


In [34]:
# зафксируем random_state
random_state = 42
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'C': float(params['C']),
       'solver': str(params['solver'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # кросс-валидация
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров

%time
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))
if best['solver'] == 0:
    best['solver'] = 'liblinear'
else:
    best['solver'] ='saga'
# рассчитаем точность для тестовой выборки
model_rf = linear_model.LogisticRegression(
    random_state=random_state, 
    C=float(best['LR_C']),
    solver=best['solver'],
    max_iter=30
)
model_rf.fit(X_train, y_train)
y_train_pred = model_rf.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на валидационном наборе: {:.2f}".format(model_rf.score(X_valid, y_valid)))
y_test_pred = model_rf.predict(X_valid)
print('f1_score на валидационном наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))


CPU times: total: 0 ns
Wall time: 0 ns
  5%|▌         | 1/20 [00:03<01:11,  3.75s/trial, best loss: -0.7672135159429093]

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 15%|█▌        | 3/20 [00:14<01:20,  4.72s/trial, best loss: -0.7747269034654849]

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 25%|██▌       | 5/20 [00:25<01:09,  4.66s/trial, best loss: -0.7747269034654849]

...

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 90%|█████████ | 18/20 [01:35<00:12,  6.33s/trial, best loss: -0.7860879406542405]

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [01:44<00:00,  5.23s/trial, best loss: -0.7860879406542405]
Наилучшие значения гиперпараметров {'LR_C': 0.08175332859022828, 'solver': 1}
f1_score на обучающем наборе: 0.85
accuracy на валидационном наборе: 0.76
f1_score на валидационном наборе: 0.78


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### <center> Optuna

In [27]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_1 = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_1.optimize(optuna_rf, n_trials=20)


# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_1.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_1.best_value))



from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

best_params = study_1.best_params
f1_train = []
f1_valid = []
for train_index, valid_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    model = ensemble.RandomForestClassifier(random_state=random_state,
                          **best_params)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    f1_train.append(metrics.f1_score(y_train, y_train_pred))
    f1_valid.append(metrics.f1_score(y_valid, y_valid_pred))
f1_train = np.array(f1_train)
f1_valid = np.array(f1_valid)
print("mean f1 score is on cross validation: {}".format(f1_valid.mean()))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_1.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_valid, y_valid)))
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

[I 2023-04-27 22:19:27,728] A new study created in memory with name: RandomForestClassifier
[I 2023-04-27 22:19:38,613] Trial 0 finished with value: 0.9456066945606695 and parameters: {'n_estimators': 112, 'max_depth': 20, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9456066945606695.
[I 2023-04-27 22:19:52,185] Trial 1 finished with value: 0.8919402985074627 and parameters: {'n_estimators': 150, 'max_depth': 11, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9456066945606695.
[I 2023-04-27 22:20:06,084] Trial 2 finished with value: 0.9206443914081146 and parameters: {'n_estimators': 143, 'max_depth': 17, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9456066945606695.
[I 2023-04-27 22:20:24,591] Trial 3 finished with value: 0.9086595492289443 and parameters: {'n_estimators': 198, 'max_depth': 20, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9456066945606695.
[I 2023-04-27 22:20:38,074] Trial 4 finished with value: 0.9912413168227122 and parameters: {'n_

Наилучшие значения гиперпараметров {'n_estimators': 132, 'max_depth': 22, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99
mean f1 score is on cross validation: 0.8151690879348843
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.79


In [30]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  C = trial.suggest_float('C', 0, 1)
  solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
  

  # создаем модель
  model = linear_model.LogisticRegression(C=C,
                                          solver=solver,
                                          max_iter=30,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_2 = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_2.optimize(optuna_lr, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_2.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_2.best_value))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_1.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_valid, y_valid)))
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

[I 2023-04-27 22:27:11,956] A new study created in memory with name: LogisticRegression
c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-04-27 22:27:13,260] Trial 0 finished with value: 0.8610451306413303 and parameters: {'C': 0.6955977683375537, 'solver': 'saga'}. Best is trial 0 with value: 0.8610451306413303.
[I 2023-04-27 22:27:13,669] Trial 1 finished with value: 0.881508078994614 and parameters: {'C': 0.536018828751838, 'solver': 'liblinear'}. Best is trial 1 with value: 0.881508078994614.
[I 2023-04-27 22:27:14,068] Trial 2 finished with value: 0.883428228948157 and parameters: {'C': 0.5778909318194105, 'solver': 'liblinear'}. Best is trial 2 with value: 0.883428228948157.
c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was 

Наилучшие значения гиперпараметров {'C': 0.9879047702092318, 'solver': 'liblinear'}
f1_score на обучающем наборе: 0.89
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.79


Изходя из полученных результатов можно сделать вывод что метод случайного леса на продвинутых методах переобучается, следовательно нужно тчательнее подойти к выборам параметров, то есть поиграть с диапозонома